# VISION ZERO HACKATHON
The purpose of this notebook is to import the TXDOT crash data, and clean and limit the columns (called limit_df).  A secondary dataset was created by aggregating the crash data at the level of intersections (called crash_agg_df).  

After the logistic regression model (Ran by Ben in R) and random forest model (Ran by Xin in R) were run, the predictions were added, and a Risk Rating for each intersection was assigned based on the random forest prediction (ULTRA HIGH, HIGH, MODERATE, MILD, LOW).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime 
import pickle

from IPython.display import display
pd.options.display.max_columns = None

In [2]:
#Import the data
data_dir = "/home/julie/koupon-media/notebooks/julie_notebooks/python_practice/vision_zero/data/crash_data/"
data_files = os.listdir(data_dir)
data_files = [file for file in data_files if "txdot" in file]

In [3]:
data_files

['txdot_cris_crashes_hgac_201804_201806.csv',
 'txdot_cris_crashes_hgac_201807_201808.csv',
 'txdot_cris_crashes_hgac_201809.csv',
 'txdot_cris_crashes_hgac_201810_201901.csv',
 'txdot_cris_crashes_hgac_201504_201803.csv']

In [4]:
#Find columns that are different among files to rename for uniformity when concatenating dataframes
col0 = list(pd.read_csv(data_dir+data_files[0]).columns)
col1 = list(pd.read_csv(data_dir+data_files[1]).columns)
col2 = list(pd.read_csv(data_dir+data_files[2]).columns)
col3 = list(pd.read_csv(data_dir+data_files[3]).columns)
col4 = list(pd.read_csv(data_dir+data_files[4]).columns)

print(np.sort(list(set(list(col0))
 .symmetric_difference(list(col1)))))

print(np.sort(list(set(list(col0))
 .symmetric_difference(list(col2)))))

print(np.sort(list(set(list(col0))
 .symmetric_difference(list(col3)))))

print(np.sort(list(set(list(col0))
 .symmetric_difference(list(col4)))))

print(np.sort(list(set(list(col1))
 .symmetric_difference(list(col2)))))

print(np.sort(list(set(list(col1))
 .symmetric_difference(list(col3)))))

print(np.sort(list(set(list(col1))
 .symmetric_difference(list(col4)))))

print(np.sort(list(set(list(col2))
 .symmetric_difference(list(col3)))))

print(np.sort(list(set(list(col2))
 .symmetric_difference(list(col4)))))

print(np.sort(list(set(list(col3))
 .symmetric_difference(list(col4)))))


/data/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (26,78,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/data/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (26,78,100,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/data/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (26,43,78,100,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['file']
['file' 'month' 'year']
['CrashIncapacitatingInjuryCount' 'CrashNonSuspectedSeriousInjuryCount'
 'CrashNonincapacitatingInjuryCount' 'CrashSuspectedSeriousInjuryCount'
 'file' 'month' 'year']
['file']
['month' 'year']
['CrashIncapacitatingInjuryCount' 'CrashNonSuspectedSeriousInjuryCount'
 'CrashNonincapacitatingInjuryCount' 'CrashSuspectedSeriousInjuryCount'
 'month' 'year']
[]
['CrashIncapacitatingInjuryCount' 'CrashNonSuspectedSeriousInjuryCount'
 'CrashNonincapacitatingInjuryCount' 'CrashSuspectedSeriousInjuryCount']
['month' 'year']
['CrashIncapacitatingInjuryCount' 'CrashNonSuspectedSeriousInjuryCount'
 'CrashNonincapacitatingInjuryCount' 'CrashSuspectedSeriousInjuryCount'
 'month' 'year']


In [5]:
#Load and combine all of the csvs into a single df
#Rename the two columns that have different names between files, as found by previous code block
agg_df = pd.read_csv(data_dir+data_files[0])

for file in data_files[1::]:
    df = pd.read_csv(data_dir+file)
    if "CrashNonSuspectedSeriousInjuryCount" in list(df.columns):
        df = df.rename(index = str, 
                       columns = {"CrashNonSuspectedSeriousInjuryCount":"CrashNonincapacitatingInjuryCount"})
    if "CrashSuspectedSeriousInjuryCount" in list(df.columns):
        df = df.rename(index = str, 
                       columns = {"CrashSuspectedSeriousInjuryCount":"CrashIncapacitatingInjuryCount"})
    agg_df = pd.concat([agg_df, df])

/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [130]:
#List of columns to keep for downstream analysis, as determined by Lauren and Kelsey

keep_cols = ['CrashID',
'StreetName',
'IntersectingStreetName',
'Latitude',
'Longitude',
'County', #filter for Harris, Montgomery, Fort Bend
'IntersectionRelated', # filter for intersection or intersection related
'SpeedLimit',
'CrashSeverity',
'NumberofLanes',
'NumberofEnteringRoads',
'TrafficControlType',
'RoadClass',
'RoadbedWidth',
'RoadwayAlignment',
'RoadwayFunction',
'RoadwayRelation',
'RoadwayPart',
'RoadwayType',
'LightCondition',
'FirstHarmfulEvent',
'n_bike',
'n_cars',
'n_peds',
'n_train',
'MedianWidth']

limit_df = agg_df.loc[:,keep_cols]

#Filter the dataframe by county (afterwards, don't need this column)
limit_df = limit_df.loc[(limit_df['County'].str.contains('Fort Bend')) |
                       (limit_df['County'].str.contains('Harris')) |
                       (limit_df['County'].str.contains('Montgomery'))]

#Filter the dataframe to only have intersection and intersection related rows (afterwards, don't need this column)
limit_df = limit_df.loc[((limit_df['IntersectionRelated'].str.contains('Intersection')) |
                       (limit_df['IntersectionRelated'].str.contains('Intersection Related'))) &
                       ~(limit_df['IntersectionRelated'].str.contains('Non Intersection'))]

#Data Cleaning
#SpeedLimit - turn all negatives into NaN
#Round all numbers with %5 != 0
limit_df.loc[limit_df['SpeedLimit']<0, 'SpeedLimit'] = np.nan
limit_df.loc[limit_df['SpeedLimit']%5 != 0, 'SpeedLimit'] = round(limit_df.loc[limit_df['SpeedLimit']%5 != 0, 'SpeedLimit'],-1)

#Clean Severity
limit_df['CrashSeverity'] = limit_df['CrashSeverity'].str.upper()
limit_df.loc[limit_df['CrashSeverity']=='N - NOT INJURED', 'CrashSeverity'] = "NOT INJURED"
limit_df.loc[limit_df['CrashSeverity']=='C - POSSIBLE INJURY', 'CrashSeverity'] = "POSSIBLE INJURY"
limit_df.loc[limit_df['CrashSeverity']=='99 - UNKNOWN', 'CrashSeverity'] = "UNKNOWN"
limit_df.loc[limit_df['CrashSeverity']=='B - NON-INCAPACITATING INJURY', 'CrashSeverity'] = "NON-INCAPACITATING INJURY"
limit_df.loc[limit_df['CrashSeverity']=='A - SUSPECTED SERIOUS INJURY', 'CrashSeverity'] = "SUSPECTED SERIOUS INJURY"
limit_df.loc[limit_df['CrashSeverity']=='K - KILLED', 'CrashSeverity'] = "KILLED"

#Create a binary for CrashSeverity based on injured vs non-injured, and excluding all unknown
limit_df['CrashSeverity_Binary'] = "INJURED"
limit_df.loc[(limit_df['CrashSeverity']=='NOT INJURED'), 'CrashSeverity_Binary'] = "NON-INJURED"
limit_df = limit_df.loc[~(limit_df['CrashSeverity']=="UNKNOWN")]

#Clean Street Names
limit_df['StreetName'] = limit_df['StreetName'].str.upper()
limit_df['IntersectingStreetName'] = limit_df['IntersectingStreetName'].str.upper()

#RoadbedWidth - all No Data and nan to NaN
limit_df.loc[limit_df['RoadbedWidth']=='No Data', 'RoadbedWidth'] = np.nan
limit_df.loc[limit_df['RoadbedWidth']=='nan', 'RoadbedWidth'] = np.nan
limit_df['RoadbedWidth'] = limit_df['RoadbedWidth'].astype(float)

#NumberofLanes - all No Data and nan to NaN
limit_df.loc[limit_df['NumberofLanes']=='No Data', 'NumberofLanes'] = np.nan
limit_df.loc[limit_df['NumberofLanes']=='nan', 'NumberofLanes'] = np.nan
limit_df['NumberofLanes'] = limit_df['NumberofLanes'].astype(float)

#RoadwayType - all No Data and nan to NaN
limit_df.loc[limit_df['RoadwayType']=='No Data', 'RoadwayType'] = np.nan
limit_df.loc[limit_df['RoadwayType']=='nan', 'RoadwayType'] = np.nan

#MedianWidth - all No Data and nan to NaN
limit_df.loc[limit_df['MedianWidth']=='No Data', 'MedianWidth'] = np.nan
limit_df.loc[limit_df['MedianWidth']=='nan', 'MedianWidth'] = np.nan
limit_df['MedianWidth'] = limit_df['MedianWidth'].astype(float)

#Rounding lat lng
limit_df['Latitude'] = round(limit_df['Latitude'],3)
limit_df['Longitude'] = round(limit_df['Longitude'],3)

#Create unique intersection ID to aggregate crashes on
limit_df['IntersectionID_1'] = limit_df['StreetName']+" AND "+limit_df['IntersectingStreetName']
limit_df['IntersectionID_2'] = limit_df['IntersectingStreetName']+" AND "+limit_df['StreetName']

intersection_id1 = list(limit_df['IntersectionID_1'].unique())
intersection_id2 = list(limit_df['IntersectionID_2'].unique())
limit_df['IntersectionID'] = limit_df['IntersectionID_1']

limit_df.loc[limit_df['IntersectionID_2'].isin(intersection_id1),
             'IntersectionID'] = limit_df.loc[limit_df['IntersectionID_2'].isin(intersection_id1),
                                                                                                    'IntersectionID_2']


In [131]:
#Limit to columns of interest
limit_df = limit_df.loc[:,['IntersectionID',
                           'CrashID',
                           'Latitude',
                           'Longitude',
                          'SpeedLimit',
       'CrashSeverity', 'NumberofLanes', 'NumberofEnteringRoads',
       'TrafficControlType', 'RoadClass', 'RoadbedWidth', 'RoadwayAlignment',
       'RoadwayFunction', 'RoadwayRelation', 'RoadwayPart', 'RoadwayType',
       'LightCondition', 'FirstHarmfulEvent', 'n_bike', 'n_cars', 'n_peds',
       'n_train', 'MedianWidth', 'CrashSeverity_Binary']]

limit_df = limit_df.sort_values('IntersectionID')
limit_df = limit_df.reset_index(drop = True)

In [132]:
# #Look at the number of null rows for all columns
# limit_df.isnull().sum()

In [134]:
#Adding Kelsey's data
transitstops = pd.read_csv(data_dir+'intersections_transitstops_xw.csv')

transitstops = transitstops.loc[:,['IntersectionID',
                                  'transit_n_stops',
                                  'transit_n_trips',
                                  'transit_n_routes']]

sidewalks = pd.read_csv(data_dir+'intersections_sidewalks_xw.csv')

limit_df = pd.merge(limit_df, transitstops, how = "left", on = "IntersectionID")
limit_df = pd.merge(limit_df, sidewalks, how = "left", on = "IntersectionID")

In [136]:
# limit_df.head()

In [137]:
#Aggregations - First step, make dummy variables for categorical data
#Rename to have the original column name included
#This totally could have been done in a function, but I was tired
CrashSeverity_dummies = pd.get_dummies(limit_df['CrashSeverity'])
CrashSeverity_dummies.columns = ['CrashSeverity_'+col for col in CrashSeverity_dummies.columns.values]

NumberofEnteringRoads_dummies = pd.get_dummies(limit_df['NumberofEnteringRoads'])
NumberofEnteringRoads_dummies.columns = ['NumberofEnteringRoads_'+col for col in NumberofEnteringRoads_dummies.columns.values]

TrafficControlType_dummies = pd.get_dummies(limit_df['TrafficControlType'])
TrafficControlType_dummies.columns = ['TrafficControlType_'+col for col in TrafficControlType_dummies.columns.values]

RoadClass_dummies = pd.get_dummies(limit_df['RoadClass'])
RoadClass_dummies.columns = ['RoadClass_'+col for col in RoadClass_dummies.columns.values]

RoadwayAlignment_dummies = pd.get_dummies(limit_df['RoadwayAlignment'])
RoadwayAlignment_dummies.columns = ['RoadwayAlignment_'+col for col in RoadwayAlignment_dummies.columns.values]

RoadwayFunction_dummies = pd.get_dummies(limit_df['RoadwayFunction'])
RoadwayFunction_dummies.columns = ['RoadwayFunction_'+col for col in RoadwayFunction_dummies.columns.values]

RoadwayRelation_dummies = pd.get_dummies(limit_df['RoadwayRelation'])
RoadwayRelation_dummies.columns = ['RoadwayRelation_'+col for col in RoadwayRelation_dummies.columns.values]

RoadwayPart_dummies = pd.get_dummies(limit_df['RoadwayPart'])
RoadwayPart_dummies.columns = ['RoadwayPart_'+col for col in RoadwayPart_dummies.columns.values]

LightCondition_dummies = pd.get_dummies(limit_df['LightCondition'])
LightCondition_dummies.columns = ['LightCondition_'+col for col in LightCondition_dummies.columns.values]

FirstHarmfulEvent_dummies = pd.get_dummies(limit_df['FirstHarmfulEvent'])
FirstHarmfulEvent_dummies.columns = ['FirstHarmfulEvent_'+col for col in FirstHarmfulEvent_dummies.columns.values]

RoadwayType_dummies = pd.get_dummies(limit_df['RoadwayType'])
RoadwayType_dummies.columns = ['RoadwayType_'+col for col in RoadwayType_dummies.columns.values]

CrashSeverity_Binary_dummies = pd.get_dummies(limit_df['CrashSeverity_Binary'])
CrashSeverity_Binary_dummies.columns = ['CrashSeverity_Binary_'+col for col in CrashSeverity_Binary_dummies.columns.values]


In [162]:
limit_df.head()

,IntersectionID,CrashID,Latitude,Longitude,SpeedLimit,CrashSeverity,NumberofLanes,NumberofEnteringRoads,TrafficControlType,RoadClass,RoadbedWidth,RoadwayAlignment,RoadwayFunction,RoadwayRelation,RoadwayPart,RoadwayType,LightCondition,FirstHarmfulEvent,n_bike,n_cars,n_peds,n_train,MedianWidth,CrashSeverity_Binary,transit_n_stops,transit_n_trips,transit_n_routes,SidewalkFt_100Ft,SidewalkBinary
0,10 AND W LOOP FWY S,15144049,29.758,-95.455,35.0,NON-INCAPACITATING INJURY,NaN,Other (Explain In Narrative),Yield Sign,Interstate,NaN,"Straight, Level",NaN,On Roadway,Service/Frontage Road,NaN,"Dark, Lighted",Pedestrian,0,1,1,0,NaN,INJURED,0,0,0,184.545097,1
1,10 AND W LOOP FWY S,15497736,29.758,-95.455,30.0,NOT INJURED,NaN,Not Applicable,None,Interstate,NaN,"Straight, Level",NaN,Median,Service/Frontage Road,NaN,"Dark, Lighted",Fixed Object,0,1,0,0,NaN,NON-INJURED,0,0,0,184.545097,1
2,10 AND W LOOP FWY S,14959704,29.758,-95.455,35.0,NOT INJURED,NaN,Four Entering Roads,Signal Light,Interstate,NaN,"Straight, Level",NaN,On Roadway,Service/Frontage Road,NaN,Daylight,Motor Vehicle In Transport,0,2,0,0,NaN,NON-INJURED,0,0,0,184.545097,1
3,10 AND W LOOP FWY S,16741010,29.758,-95.455,45.0,NOT INJURED,NaN,Four Entering Roads,Signal Light,Interstate,NaN,"Straight, Level",NaN,On Roadway,Service/Frontage Road,NaN,Daylight,Motor Vehicle In Transport,0,2,0,0,NaN,NON-INJURED,0,0,0,184.545097,1
4,10TH ST AND AVENUE D,15441196,29.795,-95.825,30.0,NOT INJURED,NaN,Four Entering Roads,Stop Sign,City Street,NaN,"Straight, Level",NaN,On Roadway,Main/Proper Lane,NaN,Daylight,Motor Vehicle In Transport,0,2,0,0,NaN,NON-INJURED,0,0,0,181.120685,1


In [163]:
# Save the limited cleaned dataset (at the crash level, not aggregated to intersections)
# Good for making visualizations
limit_df.to_csv('updated_limited_clean_txdot_dataset.csv')

In [180]:
limit_df.head()

,IntersectionID,CrashID,Latitude,Longitude,SpeedLimit,CrashSeverity,NumberofLanes,NumberofEnteringRoads,TrafficControlType,RoadClass,RoadbedWidth,RoadwayAlignment,RoadwayFunction,RoadwayRelation,RoadwayPart,RoadwayType,LightCondition,FirstHarmfulEvent,n_bike,n_cars,n_peds,n_train,MedianWidth,CrashSeverity_Binary,transit_n_stops,transit_n_trips,transit_n_routes,SidewalkFt_100Ft,SidewalkBinary
0,10 AND W LOOP FWY S,15144049,29.758,-95.455,35.0,NON-INCAPACITATING INJURY,NaN,Other (Explain In Narrative),Yield Sign,Interstate,NaN,"Straight, Level",NaN,On Roadway,Service/Frontage Road,NaN,"Dark, Lighted",Pedestrian,0,1,1,0,NaN,INJURED,0,0,0,184.545097,1
1,10 AND W LOOP FWY S,15497736,29.758,-95.455,30.0,NOT INJURED,NaN,Not Applicable,None,Interstate,NaN,"Straight, Level",NaN,Median,Service/Frontage Road,NaN,"Dark, Lighted",Fixed Object,0,1,0,0,NaN,NON-INJURED,0,0,0,184.545097,1
2,10 AND W LOOP FWY S,14959704,29.758,-95.455,35.0,NOT INJURED,NaN,Four Entering Roads,Signal Light,Interstate,NaN,"Straight, Level",NaN,On Roadway,Service/Frontage Road,NaN,Daylight,Motor Vehicle In Transport,0,2,0,0,NaN,NON-INJURED,0,0,0,184.545097,1
3,10 AND W LOOP FWY S,16741010,29.758,-95.455,45.0,NOT INJURED,NaN,Four Entering Roads,Signal Light,Interstate,NaN,"Straight, Level",NaN,On Roadway,Service/Frontage Road,NaN,Daylight,Motor Vehicle In Transport,0,2,0,0,NaN,NON-INJURED,0,0,0,184.545097,1
4,10TH ST AND AVENUE D,15441196,29.795,-95.825,30.0,NOT INJURED,NaN,Four Entering Roads,Stop Sign,City Street,NaN,"Straight, Level",NaN,On Roadway,Main/Proper Lane,NaN,Daylight,Motor Vehicle In Transport,0,2,0,0,NaN,NON-INJURED,0,0,0,181.120685,1


In [146]:
#Create a new dataframe that has the categorical variables now all as dummy variables
crash = limit_df.loc[:,['IntersectionID',
                        'CrashID',
                        'SpeedLimit',
                        'RoadbedWidth', 
                        'NumberofLanes',
                        'MedianWidth',
                        'SidewalkFt_100Ft',
                        'SidewalkBinary',
                        'n_bike', 
                        'n_cars',
                        'n_peds', 
                        'n_train',
                        'transit_n_stops',
                        'transit_n_trips', 
                        'transit_n_routes']]

crash = pd.concat([crash, 
                  CrashSeverity_dummies,
                  NumberofEnteringRoads_dummies,
                  TrafficControlType_dummies,
                  RoadClass_dummies,
                  RoadwayAlignment_dummies,
                  RoadwayFunction_dummies,
                  RoadwayRelation_dummies,
                  RoadwayPart_dummies,
                  LightCondition_dummies,
                  FirstHarmfulEvent_dummies,
                  RoadwayType_dummies,
                  CrashSeverity_Binary_dummies], axis = 1)

In [147]:
#Create a column for the number of crashes per intersection
crash_num = crash.loc[:,['IntersectionID',
            'CrashID']].groupby(['IntersectionID']).count().reset_index()
crash_num = crash_num.rename(index = str, columns = {'CrashID':'crash_num'})

#Create a dataframe for the aggregated crash data by intersection
#Probably should have named this variable intersection_df, but oh well
crash_agg_df = crash.loc[:,['IntersectionID']]
crash_agg_df = pd.merge(crash_agg_df, crash_num, how = "left", on = "IntersectionID")

In [148]:
#Functions to aggregate the continuous and categorical variable columns
#Continuous - get min, max, mean, and sum (not all are relevant to all variables)
#Categorical - get mean (which is essentially a proportion, good for modeling)

def continuous_agg(df, agg_col):
    grouped_df = df.loc[:,['IntersectionID',
            agg_col]].groupby(['IntersectionID']).agg([np.min, np.max, np.mean, np.sum])
    grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
    grouped_df = grouped_df.reset_index()
    return grouped_df

def categorical_agg(df, agg_col):
    grouped_df = df.loc[:,['IntersectionID',
            agg_col]].groupby(['IntersectionID']).agg([np.mean])
    grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
    grouped_df = grouped_df.reset_index()
    return grouped_df

#Run through the continuous variables and run the continuous aggregator
for col in crash.columns[2:6]:
    print(f"Now doing {col}")
    agged_df = continuous_agg(crash, col)
    crash_agg_df = pd.merge(crash_agg_df, agged_df, how = "left", on = "IntersectionID")

#Run through the categorical variables and get the means    
for col in crash.columns[6::]:
    print(f"Now doing {col}")
    agged_df = categorical_agg(crash, col)
    crash_agg_df = pd.merge(crash_agg_df, agged_df, how = "left", on = "IntersectionID")

Now doing SpeedLimit
Now doing RoadbedWidth
Now doing NumberofLanes
Now doing MedianWidth
Now doing SidewalkFt_100Ft
Now doing SidewalkBinary
Now doing n_bike
Now doing n_cars
Now doing n_peds
Now doing n_train
Now doing transit_n_stops
Now doing transit_n_trips
Now doing transit_n_routes
Now doing CrashSeverity_INCAPACITATING INJURY
Now doing CrashSeverity_KILLED
Now doing CrashSeverity_NON-INCAPACITATING INJURY
Now doing CrashSeverity_NOT INJURED
Now doing CrashSeverity_POSSIBLE INJURY
Now doing CrashSeverity_SUSPECTED SERIOUS INJURY
Now doing NumberofEnteringRoads_Cloverleaf
Now doing NumberofEnteringRoads_Five Entering Roads
Now doing NumberofEnteringRoads_Four Entering Roads
Now doing NumberofEnteringRoads_Not Applicable
Now doing NumberofEnteringRoads_Other (Explain In Narrative)
Now doing NumberofEnteringRoads_Six Entering Roads
Now doing NumberofEnteringRoads_Three Entering Roads - T
Now doing NumberofEnteringRoads_Three Entering Roads - Y
Now doing NumberofEnteringRoads_Traffi

In [149]:
#Drop all duplicate rows
crash_agg_df = crash_agg_df.drop_duplicates()

#Save the new intersection crash aggregated 
crash_agg_df.to_csv('intersection_dataset.csv')

In [150]:
crash_agg_df.head()

,IntersectionID,crash_num,SpeedLimit_amin,SpeedLimit_amax,SpeedLimit_mean,SpeedLimit_sum,RoadbedWidth_amin,RoadbedWidth_amax,RoadbedWidth_mean,RoadbedWidth_sum,NumberofLanes_amin,NumberofLanes_amax,NumberofLanes_mean,NumberofLanes_sum,MedianWidth_amin,MedianWidth_amax,MedianWidth_mean,MedianWidth_sum,SidewalkFt_100Ft_mean,SidewalkBinary_mean,n_bike_mean,n_cars_mean,n_peds_mean,n_train_mean,transit_n_stops_mean,transit_n_trips_mean,transit_n_routes_mean,CrashSeverity_INCAPACITATING INJURY_mean,CrashSeverity_KILLED_mean,CrashSeverity_NON-INCAPACITATING INJURY_mean,CrashSeverity_NOT INJURED_mean,CrashSeverity_POSSIBLE INJURY_mean,CrashSeverity_SUSPECTED SERIOUS INJURY_mean,NumberofEnteringRoads_Cloverleaf_mean,NumberofEnteringRoads_Five Entering Roads_mean,NumberofEnteringRoads_Four Entering Roads_mean,NumberofEnteringRoads_Not Applicable_mean,NumberofEnteringRoads_Other (Explain In Narrative)_mean,NumberofEnteringRoads_Six Entering Roads_mean,NumberofEnteringRoads_Three Entering Roads - T_mean,NumberofEnteringRoads_Three Entering Roads - Y_mean,NumberofEnteringRoads_Traffic Circle_mean,TrafficControlType_Bike Lane_mean,TrafficControlType_Center Stripe/Divider_mean,TrafficControlType_Crosswalk_mean,TrafficControlType_Flagman_mean,TrafficControlType_Flashing Red Light_mean,TrafficControlType_Flashing Yellow Light_mean,TrafficControlType_Inoperative (Explain In Narrative)_mean,TrafficControlType_Marked Lanes_mean,TrafficControlType_No Passing Zone_mean,TrafficControlType_None_mean,TrafficControlType_Officer_mean,TrafficControlType_Other (Explain In Narrative)_mean,TrafficControlType_RR Gate/Signal_mean,TrafficControlType_Signal Light_mean,TrafficControlType_Signal Light With Red Light Running Camera_mean,TrafficControlType_Stop Sign_mean,TrafficControlType_Warning Sign_mean,TrafficControlType_Yield Sign_mean,RoadClass_City Street_mean,RoadClass_County Road_mean,RoadClass_Farm To Market_mean,RoadClass_Interstate_mean,RoadClass_Other Roads_mean,RoadClass_Toll Bridges_mean,RoadClass_Tollway_mean,RoadClass_US & State Highways_mean,"RoadwayAlignment_Curve, Grade_mean","RoadwayAlignment_Curve, Hillcrest_mean","RoadwayAlignment_Curve, Level_mean",RoadwayAlignment_Other (Explain In Narrative)_mean,"RoadwayAlignment_Straight, Grade_mean","RoadwayAlignment_Straight, Hillcrest_mean","RoadwayAlignment_Straight, Level_mean",RoadwayAlignment_Unknown_mean,RoadwayFunction_No Data_mean,RoadwayFunction_Rural Interstate_mean,RoadwayFunction_Rural Major Coll_mean,RoadwayFunction_Rural Minor Arterial_mean,RoadwayFunction_Rural Minor Coll_mean,RoadwayFunction_Rural Prin Arterial_mean,RoadwayFunction_Urban Collector_mean,RoadwayFunction_Urban Minor Arterial_mean,RoadwayFunction_Urban Prin Arterial (IH)_mean,RoadwayFunction_Urban Prin Arterial (Other Freeway)_mean,RoadwayFunction_Urban Prin Arterial (Other)_mean,RoadwayRelation_Median_mean,RoadwayRelation_Not Applicable_mean,RoadwayRelation_Off Roadway_mean,RoadwayRelation_On Roadway_mean,RoadwayRelation_Shoulder_mean,RoadwayPart_Connector/Flyover_mean,RoadwayPart_Entrance/On Ramp_mean,RoadwayPart_Exit/Off Ramp_mean,RoadwayPart_Main/Proper Lane_mean,RoadwayPart_Other (Explain In Narrative)_mean,RoadwayPart_Service/Frontage Road_mean,"LightCondition_Dark, Lighted_mean","LightCondition_Dark, Not Lighted_mean","LightCondition_Dark, Unknown Lighting_mean",LightCondition_Dawn_mean,LightCondition_Daylight_mean,LightCondition_Dusk_mean,LightCondition_Other (Explain In Narrative)_mean,LightCondition_Unknown_mean,FirstHarmfulEvent_Animal_mean,FirstHarmfulEvent_Fixed Object_mean,FirstHarmfulEvent_Motor Vehicle In Transport_mean,FirstHarmfulEvent_Other Non Collision_mean,FirstHarmfulEvent_Other Object_mean,FirstHarmfulEvent_Overturned_mean,FirstHarmfulEvent_Parked Car_mean,FirstHarmfulEvent_Pedalcyclist_mean,FirstHarmfulEvent_Pedestrian_mean,FirstHarmfulEvent_RR Train_mean,"RoadwayType_2 Lane, 2 Way_mean","RoadwayType_4 Or More Lanes, Divided_mean","RoadwayType_4 Or More Lanes, Undivided_mean",RoadwayType_Other Road

In [154]:
#Remove columns that are not useful for modeling
crash_agg_df_to_model = crash_agg_df.drop(["IntersectionID",
          "SpeedLimit_amax","SpeedLimit_sum",
          "NumberofLanes_amin","NumberofLanes_amax", "NumberofLanes_mean",
          "NumberofLanes_sum", "RoadbedWidth_amin","RoadbedWidth_amax",
          "RoadbedWidth_mean", "RoadbedWidth_sum","CrashSeverity_INCAPACITATING INJURY_mean",
          "CrashSeverity_KILLED_mean","CrashSeverity_NON-INCAPACITATING INJURY_mean",
          "CrashSeverity_NOT INJURED_mean","CrashSeverity_POSSIBLE INJURY_mean",
          "CrashSeverity_SUSPECTED SERIOUS INJURY_mean", "CrashSeverity_Binary_NON-INJURED_mean",
          "MedianWidth_amin","MedianWidth_amax","MedianWidth_mean","MedianWidth_sum"], axis = 1)

#Save the new intersection crash aggregated for modeling
crash_agg_df_to_model.to_csv('intersection_dataset_to_model.csv', index = False)

In [157]:
#Dataframe for prediction, same as the modeling one but has the IntersectionID
crash_agg_df_to_predict = crash_agg_df.drop([
          "SpeedLimit_amax","SpeedLimit_sum",
          "NumberofLanes_amin","NumberofLanes_amax", "NumberofLanes_mean",
          "NumberofLanes_sum", "RoadbedWidth_amin","RoadbedWidth_amax",
          "RoadbedWidth_mean", "RoadbedWidth_sum","CrashSeverity_INCAPACITATING INJURY_mean",
          "CrashSeverity_KILLED_mean","CrashSeverity_NON-INCAPACITATING INJURY_mean",
          "CrashSeverity_NOT INJURED_mean","CrashSeverity_POSSIBLE INJURY_mean",
          "CrashSeverity_SUSPECTED SERIOUS INJURY_mean", "CrashSeverity_Binary_NON-INJURED_mean",
          "MedianWidth_amin","MedianWidth_amax","MedianWidth_mean","MedianWidth_sum"], axis = 1)

#Save the new intersection crash aggregated 
crash_agg_df_to_predict.to_csv('intersection_dataset_to_predict.csv', index = False)

In [244]:
# crash_agg_df_to_model.head()

In [156]:
#Investigate nulls in the modeling data set - seems to be very few, and only for speed limit.
#Consider dropping
crash_agg_df_to_model.isnull().sum()

crash_num                                                       0
SpeedLimit_amin                                               747
SpeedLimit_mean                                               747
SidewalkFt_100Ft_mean                                           0
SidewalkBinary_mean                                             0
n_bike_mean                                                     0
n_cars_mean                                                     0
n_peds_mean                                                     0
n_train_mean                                                    0
transit_n_stops_mean                                            0
transit_n_trips_mean                                            0
transit_n_routes_mean                                           0
NumberofEnteringRoads_Cloverleaf_mean                           0
NumberofEnteringRoads_Five Entering Roads_mean                  0
NumberofEnteringRoads_Four Entering Roads_mean                  0
NumberofEn

In [243]:
###RESULTS FROM LOGISTIC REGRESSION - Top predictive variables
log_reg_output = pd.read_csv(data_dir+"logistic_regression_results.csv")
log_reg_output.sort_values('Pr...z..').head(10)

,Estimate,Std..Error,z.value,Pr...z..,variable
0,0.510992,0.023652,21.604289,1.636907e-103,crash_num
29,0.632216,0.078245,8.079962,6.478691e-16,RoadClass_City.Street_mean
64,2.234794,0.458713,4.871883,1.105396e-06,FirstHarmfulEvent_Overturned_mean
2,0.071243,0.016365,4.353319,1.340916e-05,SpeedLimit_mean
5,0.421915,0.122948,3.431658,5.999028e-04,n_cars_mean
13,0.269097,0.085790,3.136705,1.708582e-03,NumberofEnteringRoads_Four.Entering.Roads_mean
65,-1.162949,0.374730,-3.103436,1.912874e-03,FirstHarmfulEvent_Parked.Car_mean
1,-0.047985,0.015535,-3.088843,2.009379e-03,SpeedLimit_amin
25,-1.350769,0.525523,-2.570332,1.016010e-02,TrafficControlType_Other..Explain.In.Narrative...
66,10.300947,4.232449,2.433803,1.494113e-02,FirstHarmfulEvent_Pedalcyclist_mean


In [174]:
#PREDICTIVE RESULTS FROM THE LOGISTIC REGRESSION - Add to intersection data frame
pred_results = pd.read_csv(data_dir+"logistic_regression_injury_prediction.csv")
pred_results = pred_results.rename(index = str, columns = {'Unnamed: 0':'IntersectionID'})
results = pd.merge(crash_agg_df_to_predict.dropna(), pred_results, how = "left", on = "IntersectionID")
results = results.sort_values(['injury_prediction', 'CrashSeverity_Binary_INJURED_mean'], ascending = False)

In [ ]:
###RESULTS FROM RANDOM FOREST

In [202]:
forest_results = pd.read_csv(data_dir+"completedataRForest.csv")
forest_results = forest_results.loc[:,['IntersectionID',
                     'RforestPredic']]
results = pd.merge(results, forest_results, how = "left", on = "IntersectionID")
results = results.sort_values("RforestPredic", ascending = False)

In [221]:
#Highest risk intersections according to the random forest
results['RiskRating'] = "NO RATING"
results.loc[(results['RforestPredic']>=0.8),'RiskRating'] = "ULTRA HIGH"

results.loc[(results['RforestPredic']<0.8) &
            (results['RforestPredic']>=0.6),'RiskRating'] = "HIGH"

results.loc[(results['RforestPredic']<0.6) &
            (results['RforestPredic']>=0.4),'RiskRating'] = "MODERATE"

results.loc[(results['RforestPredic']<0.4) &
            (results['RforestPredic']>=0.2),'RiskRating']= "MILD"

results.loc[(results['RforestPredic']<0.2) &
            (results['RforestPredic']>=0),'RiskRating']= "LOW"

In [229]:
results.head()

,IntersectionID,crash_num,SpeedLimit_amin,SpeedLimit_mean,SidewalkFt_100Ft_mean,SidewalkBinary_mean,n_bike_mean,n_cars_mean,n_peds_mean,n_train_mean,transit_n_stops_mean,transit_n_trips_mean,transit_n_routes_mean,NumberofEnteringRoads_Cloverleaf_mean,NumberofEnteringRoads_Five Entering Roads_mean,NumberofEnteringRoads_Four Entering Roads_mean,NumberofEnteringRoads_Not Applicable_mean,NumberofEnteringRoads_Other (Explain In Narrative)_mean,NumberofEnteringRoads_Six Entering Roads_mean,NumberofEnteringRoads_Three Entering Roads - T_mean,NumberofEnteringRoads_Three Entering Roads - Y_mean,NumberofEnteringRoads_Traffic Circle_mean,TrafficControlType_Bike Lane_mean,TrafficControlType_Center Stripe/Divider_mean,TrafficControlType_Crosswalk_mean,TrafficControlType_Flagman_mean,TrafficControlType_Flashing Red Light_mean,TrafficControlType_Flashing Yellow Light_mean,TrafficControlType_Inoperative (Explain In Narrative)_mean,TrafficControlType_Marked Lanes_mean,TrafficControlType_No Passing Zone_mean,TrafficControlType_None_mean,TrafficControlType_Officer_mean,TrafficControlType_Other (Explain In Narrative)_mean,TrafficControlType_RR Gate/Signal_mean,TrafficControlType_Signal Light_mean,TrafficControlType_Signal Light With Red Light Running Camera_mean,TrafficControlType_Stop Sign_mean,TrafficControlType_Warning Sign_mean,TrafficControlType_Yield Sign_mean,RoadClass_City Street_mean,RoadClass_County Road_mean,RoadClass_Farm To Market_mean,RoadClass_Interstate_mean,RoadClass_Other Roads_mean,RoadClass_Toll Bridges_mean,RoadClass_Tollway_mean,RoadClass_US & State Highways_mean,"RoadwayAlignment_Curve, Grade_mean","RoadwayAlignment_Curve, Hillcrest_mean","RoadwayAlignment_Curve, Level_mean",RoadwayAlignment_Other (Explain In Narrative)_mean,"RoadwayAlignment_Straight, Grade_mean","RoadwayAlignment_Straight, Hillcrest_mean","RoadwayAlignment_Straight, Level_mean",RoadwayAlignment_Unknown_mean,RoadwayFunction_No Data_mean,RoadwayFunction_Rural Interstate_mean,RoadwayFunction_Rural Major Coll_mean,RoadwayFunction_Rural Minor Arterial_mean,RoadwayFunction_Rural Minor Coll_mean,RoadwayFunction_Rural Prin Arterial_mean,RoadwayFunction_Urban Collector_mean,RoadwayFunction_Urban Minor Arterial_mean,RoadwayFunction_Urban Prin Arterial (IH)_mean,RoadwayFunction_Urban Prin Arterial (Other Freeway)_mean,RoadwayFunction_Urban Prin Arterial (Other)_mean,RoadwayRelation_Median_mean,RoadwayRelation_Not Applicable_mean,RoadwayRelation_Off Roadway_mean,RoadwayRelation_On Roadway_mean,RoadwayRelation_Shoulder_mean,RoadwayPart_Connector/Flyover_mean,RoadwayPart_Entrance/On Ramp_mean,RoadwayPart_Exit/Off Ramp_mean,RoadwayPart_Main/Proper Lane_mean,RoadwayPart_Other (Explain In Narrative)_mean,RoadwayPart_Service/Frontage Road_mean,"LightCondition_Dark, Lighted_mean","LightCondition_Dark, Not Lighted_mean","LightCondition_Dark, Unknown Lighting_mean",LightCondition_Dawn_mean,LightCondition_Daylight_mean,LightCondition_Dusk_mean,LightCondition_Other (Explain In Narrative)_mean,LightCondition_Unknown_mean,FirstHarmfulEvent_Animal_mean,FirstHarmfulEvent_Fixed Object_mean,FirstHarmfulEvent_Motor Vehicle In Transport_mean,FirstHarmfulEvent_Other Non Collision_mean,FirstHarmfulEvent_Other Object_mean,FirstHarmfulEvent_Overturned_mean,FirstHarmfulEvent_Parked Car_mean,FirstHarmfulEvent_Pedalcyclist_mean,FirstHarmfulEvent_Pedestrian_mean,FirstHarmfulEvent_RR Train_mean,"RoadwayType_2 Lane, 2 Way_mean","RoadwayType_4 Or More Lanes, Divided_mean","RoadwayType_4 Or More Lanes, Undivided_mean",RoadwayType_Other Road Type_mean,CrashSeverity_Binary_INJURED_mean,injury_prediction,RiskRating,RforestPredic
15848,CAPITOL ST AND S WAYSIDE DR,1,35.0,35.0,288.271183,1,1.0,1.0,0.0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
#SAVE THE OUTPUT WITH PREDICTIONS
results.to_csv('predictions_both_models.csv')

In [ ]:
###PLAYING AROUND WITH CLUSTERING - THIS PART BELOW GETS REALLY MESSY AND NOT USEFUL

In [104]:
import math
import matplotlib.pyplot as plt
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from collections import Counter
from scipy import stats
import scipy
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans

In [105]:
#clustering of intersections is not a good way to go it seems
intersection = crash_agg_df_to_model.dropna()
intersection_linkage = scipy.cluster.hierarchy.linkage(intersection, method = 'complete', metric = 'euclidean')

In [106]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)', fontsize = 20)
        plt.xlabel('sample index or (cluster size)', fontsize=18)
        plt.ylabel('distance', fontsize=18)
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center', fontsize = 16)
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

In [ ]:
plt.figure(figsize=(25,15))
fancy_dendrogram(
    intersection_linkage,
    truncate_mode='lastp',
    p=100,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,
    annotate_above=10,  # useful in small plots so annotations don't overlap
     max_d = 400
)
plt.show()